
# Assignment: Linear Regression, Logistic Regression, and K-Means (From Scratch)

**Instructions**
- You are NOT allowed to use `scikit-learn` for model implementation, scaling.
- You may use it for implementation of clustering
- You may use: `numpy`, `matplotlib`, and standard Python libraries only.
- Every step (scaling, loss, gradients, optimization) must be implemented manually.
- Clearly comment your code and explain your reasoning in Markdown cells.


## Question 1: Linear Regression from Scratch (with Standardization and Regularization)

You are given a dataset `(X, y)`.

### Tasks
1. Implement **StandardScaler manually**:
   - Compute mean and standard deviation for each feature.
   - Standardize the features.
2. Implement **Linear Regression using Gradient Descent**.
3. Add **L2 Regularization (Ridge Regression)**.
4. Plot:
   - Loss vs iterations
   - True vs predicted values

Do NOT use `sklearn`.


In [1]:

import numpy as np
import matplotlib.pyplot as plt


In [2]:

# Implement StandardScaler manually ,  first read about it, how it works and then implement it
class StandardScalerManual:
    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        self.std_ = np.std(X, axis=0)
        pass

    def transform(self, X):
        return (X - self.mean_) / self.std_
        pass

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)
        pass


In [3]:

# Implement Linear Regression from scratch, here you have to also construct the regulization term coefficient of which will be
# denoted by l2_lambda
# try to implement L1 regularization or atlease read about it and where it is used
class LinearRegressionManual:
    def __init__(self, lr=0.01, epochs=1000, l2_lambda=0.0):
        self.lr = lr
        self.epochs = epochs
        self.l2_lambda = l2_lambda
        self.theta = None
        self.loss_history = []
        pass

    def fit(self, X, y):
        self.m, self.n = X.shape
        X_b = np.c_[np.ones((self.m, 1)), X]

        # Initialize weights
        self.theta = np.random.randn(self.n + 1, 1)

        # Gradient Descent
        for i in range(self.epochs):
            y_pred = X_b.dot(self.theta)
            error = y_pred - y

            # Compute loss with L2 regularization
            loss = (1/self.m) * np.sum(error**2) + (self.l2_lambda/(2*self.m)) * np.sum(self.theta[1:]**2)
            self.loss_history.append(loss)

            # Compute gradients
            gradients = (2/self.m) * X_b.T.dot(error)
            gradients[1:] += (self.l2_lambda/self.m) * self.theta[1:]

            # Update weights
            self.theta -= self.lr * gradients
        pass

    def predict(self, X):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return X_b.dot(self.theta)
        pass



## Question 2: Logistic Regression from Scratch (with Standardization and Regularization)

You are given a binary classification dataset.

### Tasks
1. Reuse your **manual StandardScaler**.
2. Implement **Logistic Regression using Gradient Descent**.
3. Use:
   - Sigmoid function
   - Binary Cross Entropy loss
4. Add **L2 Regularization**.
5. Report:
   - Training loss curve
   - Final accuracy

Do NOT use `sklearn`.


In [5]:

#Implement sigmoid function as told in the lectures
def sigmoid(z):
    return 1 / (1 + np.exp(-z))



In [7]:

#Implement Logistic Regression from scratch and here also add the regularizaation term
class LogisticRegressionManual:
    def __init__(self, lr=0.01, epochs=1000, l2_lambda=0.0):
        self.lr = lr
        self.epochs = epochs
        self.l2_lambda = l2_lambda
        self.theta = None
        self.loss_history = []
        pass

    def fit(self, X, y):
        self.m, self.n = X.shape

        # Add bias term
        X_b = np.c_[np.ones((self.m, 1)), X]

        # Initialize weights
        self.theta = np.random.randn(self.n + 1, 1)

        # Gradient Descent
        for i in range(self.epochs):
            z = X_b.dot(self.theta)
            y_pred = sigmoid(z)
            # Compute binary cross-entropy loss with L2 regularization
            loss = (-1/self.m) * np.sum(y*np.log(y_pred + 1e-15) + (1-y)*np.log(1-y_pred + 1e-15))
            loss += (self.l2_lambda/(2*self.m)) * np.sum(self.theta[1:]**2)  # do not regularize bias
            self.loss_history.append(loss)
            # Compute gradients
            gradients = (1/self.m) * X_b.T.dot(y_pred - y)
            gradients[1:] += (self.l2_lambda/self.m) * self.theta[1:]  # L2 for weights only
            self.theta -= self.lr * gradients
        pass

    def predict_proba(self, X):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return sigmoid(X_b.dot(self.theta))
        pass

    def predict(self, X):
        return (self.predict_proba(X)).astype(int)
        pass



## Question 3: K-Means Clustering from Scratch (Matrix Clustering)

You are given a **random matrix** `M` of shape `(n, m)`.

### Tasks
Implement K-Means clustering **from scratch** such that:

1. Input:
   - A random matrix `M`
   - Number of clusters `k`
2. Output:
   - `assignment_table`: a matrix of same shape as `M`, where each element stores the **cluster label**
   - `cookbook`: a dictionary (hashmap) where:
     - Key = cluster index
     - Value = list of **positions (i, j)** belonging to that cluster
   - `centroids`: array storing centroid values

You must cluster **individual elements**, not rows.


In [8]:
import numpy as np
from sklearn.cluster import KMeans

In [9]:

# Implement K-Means for matrix elements
#CAN USE SK-LEARN FOR THIS TASK AS THIS TASK WILL HELP US DIRECTLY IN OUR PROJECT !
def kmeans_matrix(M, k, max_iters=100):
    '''
    Returns:
    assignment_table: same shape as M, contains cluster labels
    cookbook: dict -> cluster_id : list of (i, j) positions
    centroids: numpy array of centroid values
    '''
    n, m = M.shape

    # Flatten the matrix into a column vector (each element is a sample)
    M_flat = M.reshape(-1, 1)

    # Apply K-Means using scikit-learn
    kmeans = KMeans(n_clusters=k, max_iter=max_iters, random_state=0)
    kmeans.fit(M_flat)

    labels_flat = kmeans.labels_  # cluster label for each element
    centroids = kmeans.cluster_centers_.flatten()

    # Reshape labels back to matrix shape
    assignment_table = labels_flat.reshape(n, m)

    # Create cookbook dictionary
    cookbook = {i: [] for i in range(k)}
    for i in range(n):
        for j in range(m):
            cluster_id = assignment_table[i, j]
            cookbook[cluster_id].append((i, j))

    return assignment_table, cookbook, centroids
    pass



## Submission Guidelines
- Submit the completed `.ipynb` file.
- Clearly label all plots and outputs.
- Code readability and correctness matter.
- Partial credit will be given for logically correct implementations.

**Bonus**
- Compare convergence with and without standardization.
- Try different values of regularization strength.
